## Get variable from notebook

In [ ]:
from jupyter_client import BlockingKernelClient
import json
from jupyter_core.paths import jupyter_runtime_dir
import os
import glob

# Find the connection file
connection_files = glob.glob(os.path.join(jupyter_runtime_dir(), 'kernel-*.json'))
latest_connection_file = max(connection_files, key=os.path.getctime)

# Connect to the kernel
client = BlockingKernelClient()
client.load_connection_file(latest_connection_file)
client.start_channels()

# Get the value of a variable (replace 'my_variable' with your actual variable name)
variable_name = 'my_variable'
msg_id = client.execute(f"print({variable_name})")

# Get the response
while True:
    try:
        msg = client.get_iopub_msg(timeout=10)
        if msg['msg_type'] == 'stream' and msg['parent_header']['msg_id'] == msg_id:
            print(f"Value of {variable_name}: {msg['content']['text'].strip()}")
            break
    except:
        print(f"Couldn't retrieve {variable_name}")
        break

client.stop_channels()


# From `connection_info` dict

In [2]:
from jupyter_client import BlockingKernelClient

# Connection info dictionary (you already have this)
connection_info = {
    "key": "8d669f8c-24d6-4d50-b32c-d3cae7ae7b0d",
    "signature_scheme": "hmac-sha256",
    "transport": "tcp",
    "ip": "127.0.0.1",
    "hb_port": 9005,
    "control_port": 9006,
    "shell_port": 9007,
    "stdin_port": 9008,
    "iopub_port": 9009,
    "kernel_name": "spike3d-2025"
}

# Create a client and connect directly with the connection info
client = BlockingKernelClient()

# Set connection parameters directly from the dictionary
client.session.key = connection_info['key'].encode()
client.session.signature_scheme = connection_info['signature_scheme']

# Set connection addresses
client.transport = connection_info['transport']
client.ip = connection_info['ip']

# Set individual port numbers
client.hb_port = connection_info['hb_port']
client.control_port = connection_info['control_port']
client.shell_port = connection_info['shell_port']
client.stdin_port = connection_info['stdin_port']
client.iopub_port = connection_info['iopub_port']

# Start the channels
client.start_channels()

# Now you can execute code and get the value of a variable
variable_name = 't_delta'  # replace with your variable name
msg_id = client.execute(f"print({variable_name})")

# Retrieve the output
while True:
    try:
        msg = client.get_iopub_msg(timeout=10)
        if msg['msg_type'] == 'stream' and msg['parent_header']['msg_id'] == msg_id:
            print(f"Value of {variable_name}: {msg['content']['text'].strip()}")
            break
        elif msg['msg_type'] == 'error' and msg['parent_header']['msg_id'] == msg_id:
            print(f"Error: {msg['content']['evalue']}")
            break
    except Exception as e:
        print(f"Exception while retrieving variable: {e}")
        break

# Don't forget to close the connection when done
client.stop_channels()


Value of t_delta: 1211.5580800310709


In [ ]:
# For getting arbitrary results:
result_code = """
import pandas as pd
# Your custom code here
result = df.head()  # example operation
result  # will be captured as display data
"""

msg_id = client.execute(result_code)

# Get the results, including display data
while True:
    try:
        msg = client.get_iopub_msg(timeout=10)
        if msg['parent_header'].get('msg_id') == msg_id:
            # Check for different message types
            if msg['msg_type'] == 'execute_result':
                print("Result:", msg['content']['data'].get('text/plain', ''))
                break
            elif msg['msg_type'] == 'error':
                print(f"Error: {msg['content']['evalue']}")
                break
    except Exception as e:
        print(f"Exception: {e}")
        break
